# Overview

In [2]:
import subprocess
cmd = "git rev-parse --short HEAD"
hash = subprocess.check_output(cmd.split()).strip().decode('utf-8')
print(hash)

60db821


# Const

In [3]:
NB = '026'
DIR_TRAIN = './../data_ignore/input/train/'
DIR_TEST = './../data_ignore/input/test/'
DIR_WIFI = './../data_ignore/input/wifi/'
PATH_SUB = './../data_ignore/input/sample_submission.csv'
PATH_99_SUB = './../data/input/floor_99per_acc_sub.csv'
DIR_SAVE_IGNORE = f'./../data_ignore/nb/{NB}/'
DIR_SAVE = f'./../data/nb/{NB}/'

In [4]:
config_str = '''
globals:
    seed: 5713
    device: cuda
    n_label: 24
    n_splits: 5
    random_sate: 42
    lr: 0.001
    patience: 10
    epoch: 100
    batch_size: 512
    skip_evaluate_num: 5
    num_feats: 20
    t_mux: 10
'''

# Import everything I need:)

In [5]:
import os
import time
import yaml
import types
import random
import pickle
import builtins
import numpy as np
import pandas as pd
import seaborn as sns
from icecream import ic
import matplotlib.pyplot as plt
from dataclasses import dataclass
# from tqdm import tqdm
from fastprogress import progress_bar, master_bar
from glob import glob
from loguru import logger
from collections import OrderedDict

# sklearn
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import KFold

# pytorch
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F

# Function

In [6]:
def imports():
    for name, val in globals().items():
        # module imports
        if isinstance(val, types.ModuleType):
            yield name, val

            # functions / callables
        if hasattr(val, '__call__'):
            yield name, val


def noglobal(f):
    '''
    ref: https://gist.github.com/raven38/4e4c3c7a179283c441f575d6e375510c
    '''
    return types.FunctionType(f.__code__,
                              dict(imports()),
                              f.__name__,
                              f.__defaults__,
                              f.__closure__
                              )


def comp_metric(xhat, yhat, fhat, x, y, f):
    intermediate = np.sqrt(np.power(xhat-x, 2) + np.power(yhat-y, 2)) + 15 * np.abs(fhat-f)
    return intermediate.sum()/xhat.shape[0]

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [7]:
@dataclass
class ReadData:
    acce: np.ndarray
    acce_uncali: np.ndarray
    gyro: np.ndarray
    gyro_uncali: np.ndarray
    magn: np.ndarray
    magn_uncali: np.ndarray
    ahrs: np.ndarray
    wifi: np.ndarray
    ibeacon: np.ndarray
    waypoint: np.ndarray


def read_data_file(data_filename):
    acce = []
    acce_uncali = []
    gyro = []
    gyro_uncali = []
    magn = []
    magn_uncali = []
    ahrs = []
    wifi = []
    ibeacon = []
    waypoint = []

    with open(data_filename, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    for line_data in lines:
        line_data = line_data.strip()
        if not line_data or line_data[0] == '#':
            continue

        line_data = line_data.split('\t')

        if line_data[1] == 'TYPE_ACCELEROMETER':
            acce.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_ACCELEROMETER_UNCALIBRATED':
            acce_uncali.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_GYROSCOPE':
            gyro.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_GYROSCOPE_UNCALIBRATED':
            gyro_uncali.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_MAGNETIC_FIELD':
            magn.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_MAGNETIC_FIELD_UNCALIBRATED':
            magn_uncali.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_ROTATION_VECTOR':
            ahrs.append([int(line_data[0]), float(line_data[2]), float(line_data[3]), float(line_data[4])])
            continue

        if line_data[1] == 'TYPE_WIFI':
            sys_ts = line_data[0]
            ssid = line_data[2]
            bssid = line_data[3]
            rssi = line_data[4]
            lastseen_ts = line_data[6]
            wifi_data = [sys_ts, ssid, bssid, rssi, lastseen_ts]
            wifi.append(wifi_data)
            continue

        if line_data[1] == 'TYPE_BEACON':
            ts = line_data[0]
            uuid = line_data[2]
            major = line_data[3]
            minor = line_data[4]
            rssi = line_data[6]
            ibeacon_data = [ts, '_'.join([uuid, major, minor]), rssi]
            ibeacon.append(ibeacon_data)
            continue

        if line_data[1] == 'TYPE_WAYPOINT':
            waypoint.append([int(line_data[0]), float(line_data[2]), float(line_data[3])])

    acce = np.array(acce)
    acce_uncali = np.array(acce_uncali)
    gyro = np.array(gyro)
    gyro_uncali = np.array(gyro_uncali)
    magn = np.array(magn)
    magn_uncali = np.array(magn_uncali)
    ahrs = np.array(ahrs)
    wifi = np.array(wifi)
    ibeacon = np.array(ibeacon)
    waypoint = np.array(waypoint)

    return ReadData(acce, acce_uncali, gyro, gyro_uncali, magn, magn_uncali, ahrs, wifi, ibeacon, waypoint)

# Preparation

load config

In [8]:
config = yaml.safe_load(config_str)
config

{'globals': {'seed': 5713,
  'device': 'cuda',
  'n_label': 24,
  'n_splits': 5,
  'random_sate': 42,
  'lr': 0.001,
  'patience': 10,
  'epoch': 100,
  'batch_size': 512,
  'skip_evaluate_num': 5,
  'num_feats': 20,
  't_mux': 10}}

<br>

set

In [9]:
seed_everything(config['globals']['seed'])
pd.set_option('display.max_rows', 500)

if not os.path.exists(DIR_SAVE_IGNORE):
    os.makedirs(DIR_SAVE_IGNORE)
if not os.path.exists(DIR_SAVE):
    os.makedirs(DIR_SAVE)

<br>

load dataset

In [10]:
with open(f'{DIR_WIFI}train_all.pkl', 'rb') as f:
    df_train = pickle.load( f)
with open(f'{DIR_WIFI}test_all.pkl', 'rb') as f:
    df_test = pickle.load( f)

sample_submission = pd.read_csv(PATH_SUB)

# EDA

In [12]:
path_list = sorted(glob('./../data_ignore/nb/004/*'))
path_list

[]

In [35]:
n_row = 100
print('='*50)
print(f'各siteにつき、bssid {n_row}個だけ表示')
print('='*50)
for i_path, path in enumerate(path_list):
    site = path.split('_')[-1].split('.')[0]
    df = pd.read_csv(path).astype({'rssi': 'float16'})
    df_grp = df.groupby(['bssid', 'floor']).agg(cnt=('bssid', 'count'), rssi_median=('rssi', 'median'))
    
    print('-'*50)
    print(f'\t site = {site}')
    print('-'*50)
    display(df_grp.head(n_row).style.bar(subset=['cnt', 'rssi_median'], align='left', color=['red', 'green']))
    print('')

各siteにつき、bssid 100個だけ表示
--------------------------------------------------
	 site = 5a0546857ecc773753327266
--------------------------------------------------



--------------------------------------------------
	 site = 5c3c44b80379370013e0fd2b
--------------------------------------------------



--------------------------------------------------
	 site = 5d27075f03f801723c2e360f
--------------------------------------------------



--------------------------------------------------
	 site = 5d27096c03f801723c31e5e0
--------------------------------------------------



--------------------------------------------------
	 site = 5d27097f03f801723c320d97
--------------------------------------------------


...


--------------------------------------------------
	 site = 5da1389e4db8ce0c98bd0547
--------------------------------------------------



--------------------------------------------------
	 site = 5da138b74db8ce0c98bd4774
--------------------------------------------------



--------------------------------------------------
	 site = 5da958dd46f8266d0737457b
--------------------------------------------------



--------------------------------------------------
	 site = 5dbc1d84c1eb61796cf7c010
--------------------------------------------------



--------------------------------------------------
	 site = 5dc8cea7659e181adb076a3f
--------------------------------------------------


## これ大事 ssid 毎の bssid数

In [11]:
n_row = 15
for i_path, path in enumerate(path_list):
    site = path.split('_')[-1].split('.')[0]
    df = pd.read_csv(path).astype({'rssi': 'float16'})
    
    print('-'*50)
    print(f'\t site = {site}')
    print('-'*50)
    df_grp = df[['ssid ', 'bssid']].groupby('ssid ').nunique().sort_values('bssid', ascending=False)
    df_grp['change_rate'] = df[['ssid ', 'bssid']].groupby('ssid ').nunique().sort_values('bssid', ascending=False).pct_change(-1).values
    df_grp['change_rate_mean'] = df_grp['change_rate'].rolling(3, center=True, win_type='gaussian').mean(std=1 )
#     display(df_grp)
    display(df_grp.head(n_row).style.bar(subset=['bssid', 'change_rate', 'change_rate_mean'], align='left', color=['red', 'green']))
    print('')

--------------------------------------------------
	 site = 5a0546857ecc773753327266
--------------------------------------------------


,bssid,change_rate,change_rate_mean
ssid,,,
da39a3ee5e6b4b0d3255bfef95601890afd80709,543,0.491758,nan
7182afc4e5c212133d5d7d76eb3df6c24618302b,364,0.005525,0.138795
b7e6027447eb1f81327d66cfd3adbe557aabf26c,362,0.005556,0.008670
b6ffe5619e02871fcd04f61c9bb4b5c53a3f46b7,360,0.016949,0.117082
d839a45ebe64ab48b60a407d837fb01d3c0dfef9,354,0.393701,0.346296
78f32ce289cdaccfc83a01ef3cbf4af52942c063,254,0.597484,0.383152
b9f0208be00bd8b337be7f12e02e3a3ce846e22b,159,0.019231,1.934311
5731b8e08abc69d4c4d685c58164059207c93310,156,6.428571,3.115654
dbc0f004854457f59fb16ab863a3a1722cef553f,21,0.750000,2.100767



--------------------------------------------------
	 site = 5c3c44b80379370013e0fd2b
--------------------------------------------------


,bssid,change_rate,change_rate_mean
ssid,,,
44aebad0d3851fd527d792ef1af44d24f41b5799,921,0.112319,nan
e4e0e9d20827e770b65693ba9fdaefa403d5dfe4,828,1.670968,1.799055
da39a3ee5e6b4b0d3255bfef95601890afd80709,310,3.696970,2.371229
ca969a043e538dab9b1e3d3bf0e82b34c962e019,66,0.885714,1.539059
b8ea8cf3e81583f91a2d3f5253da09b1276d86ba,35,0.458333,0.449850
e46d2577d6f83617f3a8c416607f8fe1263c4483,24,0.000000,0.125615
9ee806540f418f1fffce5197aac0717792293d35,24,0.000000,0.054814
6ccdd75ff3326eb1517a988af5e5661f3bb4bdef,24,0.200000,0.273085
dbc0f004854457f59fb16ab863a3a1722cef553f,20,0.666667,0.356056



--------------------------------------------------
	 site = 5d27075f03f801723c2e360f
--------------------------------------------------


...

,bssid,change_rate,change_rate_mean
ssid,,,
da39a3ee5e6b4b0d3255bfef95601890afd80709,1465,1.474662,nan
90a13d9926e0f11cb22ac9bb4d245a64ed87411c,592,23.666667,11.110160
6cfcff54df27ca5baef9574cffb18326c7b5852d,24,0.043478,6.505937
5fb64a415514541555376c359dcf099886be5ff2,23,0.000000,0.024374
980fc4b857abe964c86ba499516fe46673dc4d68,23,0.045455,0.020539
ea1742a21aeaadc8b415d708b5eb5a47e4b780fe,22,0.000000,0.012458
f815eabb67168325255092b44a8a9861b18e1deb,22,0.000000,0.027407
9b96041fcdf03a9b1574dc75cc9d23caed9460a4,22,0.100000,0.075638
b8599529669658d73f1eba6606e3a81352e7fb80,20,0.111111,0.155919



--------------------------------------------------
	 site = 5dbc1d84c1eb61796cf7c010
--------------------------------------------------


,bssid,change_rate,change_rate_mean
ssid,,,
da39a3ee5e6b4b0d3255bfef95601890afd80709,1966,1.283391,nan
8f7d5669fb8464cd2eef926f246adaac8050b848,861,22.916667,10.881333
9f755fbf3a18018268b924f35ddeaab61072ed33,36,0.636364,6.581339
bbec6798f8898847f8f4cf2f77356ddba622aaa4,22,0.047619,0.195925
24461fb772d8b95f99df1bf82b72c050010d148e,21,0.000000,0.013051
3c2bfa511c1e8e363d08ea99efeb4b6a51d260f1,21,0.000000,0.000000
e75b5d9b518e620da0ee3d8382f2086778059dde,21,0.000000,0.000000
5c8cc1443c4b580c96f3b830c90e3d3cc257409e,21,0.000000,0.000000
b8fae28e6654aabc149345406e601e9024ee52c1,21,0.000000,0.205551



--------------------------------------------------
	 site = 5dc8cea7659e181adb076a3f
--------------------------------------------------


,bssid,change_rate,change_rate_mean
ssid,,,
da39a3ee5e6b4b0d3255bfef95601890afd80709,767,0.049248,nan
5d998a8668536c4f51004c25f474117fe9555f78,731,8.746667,4.218777
ca969a043e538dab9b1e3d3bf0e82b34c962e019,75,0.923077,2.821503
dc5c79cadaa151736d79671f33d77fafe7dcf106,39,0.026316,0.391371
b8ea8cf3e81583f91a2d3f5253da09b1276d86ba,38,0.461538,0.226727
f11066513ace56d9b2a0d049e0f186f2c5fabef5,26,0.040000,0.359907
75d4a6063cb86b0a7aecda5f00f4fb4eecc26f5e,25,0.785714,0.387080
ca91fc2f06ad6e4e122bcbf33d50bc25d24c6e92,14,0.076923,0.250098
dbc0f004854457f59fb16ab863a3a1722cef553f,13,0.000000,0.021082


# 個別チェック

In [43]:
path = glob('./../data_ignore/nb/004/*d39*')[0]
path

'./../data_ignore/nb/004/nb004_wifi_5d2709d403f801723c32bd39.csv'

In [44]:
n_row = 100
print('='*50)
print(f'bssid {n_row}個だけ表示')
print('='*50)

site = path.split('_')[-1].split('.')[0]
df = pd.read_csv(path).astype({'rssi': 'float16'})
df_grp = df.groupby(['bssid', 'floor']).agg(cnt=('bssid', 'count'), rssi_median=('rssi', 'median'))

print('-'*50)
print(f'\t site = {site}')
print('-'*50)
display(df_grp.head(n_row).style.bar(subset=['cnt', 'rssi_median'], align='left', color=['red', 'green']))
print('')

bssid 100個だけ表示
--------------------------------------------------
	 site = 5d2709d403f801723c32bd39
--------------------------------------------------
